Code taken from: https://github.com/pytorch/examples/tree/master/vae

In [12]:
from __future__ import print_function
import argparse
import os
import h5py
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [13]:
# modify to accept hard coded arguments
batch_size = 4
epochs = 2
no_cuda = False
log_interval = 10

cuda = not no_cuda and torch.cuda.is_available()

seed = 1
torch.manual_seed(seed)


# device = torch.device("cuda" if args.cuda else "cpu")
device = torch.device("cuda" if cuda else "cpu")

# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [14]:
class HydrogenDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, h5_file, root_dir):
        """
        Args:
            h5_file (string): name of the h5 file with 32 sampled cubes.
            root_dir (string): Directory with the .h5 file.
        """
        file_size = os.path.getsize(root_dir + h5_file) / 1e6 # in MBs
        print("The file size is " + str(int(file_size)) + " MBs")
        
        # self.subcubes = h5py.File('../data/sample_32.h5', 'r')
        self.subcubes = h5py.File(root_dir + h5_file, 'r')['sample32']
        self.h5_file = h5_file
        self.root_dir = root_dir

    def __len__(self):
        # Function called when len(self) is executed
        
        #print(len(self.subcubes))
        return len(self.subcubes)

    def __getitem__(self, idx):
        """
        This can be implemented in such a way that the whole h5 file read 
        using h5py.File() and get_sample() function is called to return
        a random subcube. This won't increase memory usage because the
        subcubes will be read in the same way and only the batch will
        be read into memory.
        
        Here we have implemented it so that it can be used with data
        generated by get_sample() function.
        
        The output of this function is one subcube with the dimensions
        specified by get_sample() implementation.
        """
        
        sample = self.subcubes[idx]

        return sample

In [15]:
sampled_subcubes = HydrogenDataset(h5_file="sample_32.h5",
                                    root_dir = "../data/")
sampled_subcubes

The file size is 268 MBs


In [16]:
# Data Loaders
train_loader = DataLoader(
        dataset=sampled_subcubes,
        #batch_size=args.batch_size, 
        batch_size=batch_size,
        shuffle=True, 
        **kwargs)

test_loader = DataLoader(
        dataset=sampled_subcubes,
        #batch_size=args.batch_size, 
        batch_size=batch_size,
        shuffle=True, 
        **kwargs)

In [18]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        """
        The Encoding Layers
        nn.Conv3d 
        nn.MaxPool3d 
        
        out_channels is the number of different filters we convolute 
        over the whole sampled subcube.
        
        So the first convolutional layer's in_channel should be 0 (?)
        
        In addition, the next layer's in_channel should be equal to
        the previous layer's out_channels (all examples show that
        this is the case)
        """
        
        self.encode_group1 = nn.Sequential(
            nn.Conv3d(in_channels=1, 
                      out_channels=8, 
                      kernel_size=(4,4,4),
                      stride = (2,2,2),
                      padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), 
                         stride=(2, 2, 2)))
        #init.xavier_normal(self.group1.state_dict()['weight'])
        
        self.encode_group2 = nn.Sequential(
            nn.Conv3d(in_channels = 8, 
                      out_channels = 16, 
                      kernel_size=(4,4,4),
                      stride = (2,2,2),
                      padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), 
                         stride=(2, 2, 2)))
        
        self.encode_group3 = nn.Sequential(
            nn.Conv3d(in_channels = 16, 
                      out_channels = 32, 
                      kernel_size=(4,4,4),
                      stride = (2,2,2),
                      padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), 
                         stride=(2, 2, 2)))
        
        
        """
        Fully Connected Layers after 3D Convolutional Layers
        First FC layer's input should be equal to 
        last convolutional layer's output
        8192 = 8^3 * 16 
            8^3 = (output of 2nd convolutional layer)
            16 = number of out_channels
        """
        
        self.encode_fc1 = nn.Sequential(
            nn.Linear(in_features=512, 
                      out_features=2048), 
            nn.ReLU(),
            nn.Dropout(0.5))
        #init.xavier_normal(self.fc1.state_dict()['weight'])
        
        self.encode_fc2 = nn.Sequential(
            nn.Linear(in_features = 2048,
                      out_features = 2048),
            nn.ReLU(),
            nn.Dropout(0.5))
        #init.xavier_normal(self.fc2.state_dict()['weight'])
        
        """
        The last fully connected layer's output is the dimensions
        of the embeddings?
        """
        self.encode_fc3 = nn.Sequential(
            nn.Linear(in_features=2048,
                      out_features=32))
        
        
        # Grouping convolutional and fully-connected layers in Encoding
        self._encode_conv = nn.Sequential(
            self.encode_group1,
            self.encode_group2
        )

        self._encode_fc = nn.Sequential(
            self.encode_fc1,
            self.encode_fc2
        )
        
        
        """
        The Decoding Layers
        nn.Conv3d -> nn.ConvTranspose3d
        nn.MaxPool3d -> nn.MaxUnpool3d
        """
        
        self.decode_fc1 = nn.Sequential(
            nn.Linear(in_features=32,
                      out_features=2048))
        
        self.decode_fc2 = nn.Sequential(
            nn.Linear(in_features=2048, 
                      out_features=2048), 
            nn.ReLU(),
            nn.Dropout(0.5))
        #init.xavier_normal(self.fc1.state_dict()['weight'])
        
        self.decode_fc3 = nn.Sequential(
            nn.Linear(in_features = 2048,
                      out_features = 512),
            nn.ReLU(),
            nn.Dropout(0.5))
        #init.xavier_normal(self.fc2.state_dict()['weight'])
        
        
        self.decode_group1 = nn.Sequential(
            nn.ConvTranspose3d(in_channels=32, 
                      out_channels=16, 
                      kernel_size=(4,4,4),
                      stride = (2,2,2),
                      padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxUnpool3d(kernel_size=(2, 2, 2), 
                         stride=(2, 2, 2)))
        #init.xavier_normal(self.group1.state_dict()['weight'])
        
        self.decode_group2 = nn.Sequential(
            nn.ConvTranspose3d(in_channels = 16, 
                      out_channels = 8, 
                      kernel_size=(4,4,4),
                      stride = (2,2,2),
                      padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxUnpool3d(kernel_size=(2, 2, 2), 
                         stride=(2, 2, 2)))
        
        self.decode_group3 = nn.Sequential(
            nn.ConvTranspose3d(in_channels = 8, 
                      out_channels = 1, 
                      kernel_size=(4,4,4),
                      stride = (2,2,2),
                      padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxUnpool3d(kernel_size=(2, 2, 2), 
                         stride=(2, 2, 2)))

        
        # Grouping convolutional and fully-connected layers in Decoding
        self._decode_fc = nn.Sequential(
            self.decode_fc1,
            self.decode_fc2,
            self.decode_fc3
        )
        
        self._decode_conv = nn.Sequential(
            self.decode_group1,
            self.decode_group2,
            self.decode_group3
        )


        
        
        
        
        
    # Encoding part of VAE
    def encode(self, x):
#         h1 = F.relu(self.fc1(x))
#         return self.fc21(h1), self.fc22(h1)

        out = self._encode_conv(x)
        out = out.view(out.size(0), -1)
        out = self._encode_fc(out)
        return self.encode_fc3(out)
    
    

    # Reparametrization Trick
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
    
    
    # Decoding part of VAE
    def decode(self, z):
#         h3 = F.relu(self.fc3(z))
#         return torch.sigmoid(self.fc4(h3))

        out = self._decode_fc(z)
        out = out.view(out.size(0), -1)
        out = self._decode_conv(out)
        return out
    

    # Forward Pass
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
       

In [27]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [28]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, 
                                 x.view(-1, 784), 
                                 reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [34]:
list(enumerate(train_loader))[0]

(0, tensor([[[[9.6290e-02, 1.9568e-02, 3.3024e-01,  ..., 4.3493e-01,
            3.6002e-03, 2.3939e-01],
           [3.7194e-02, 1.7360e-01, 3.9472e-01,  ..., 3.5033e-01,
            8.0818e-01, 8.6389e-01],
           [0.0000e+00, 3.6711e-02, 8.3470e-02,  ..., 4.3923e-01,
            8.6236e-01, 1.1797e+00],
           ...,
           [1.9543e-02, 2.8558e-01, 3.0122e-01,  ..., 0.0000e+00,
            1.5595e-02, 6.1377e-02],
           [0.0000e+00, 3.4958e-02, 6.2547e-02,  ..., 4.3017e-02,
            1.5451e-02, 6.0810e-02],
           [0.0000e+00, 0.0000e+00, 1.4033e-02,  ..., 5.6892e-03,
            3.9870e-02, 3.3510e-02]],
 
          [[5.3820e-01, 0.0000e+00, 0.0000e+00,  ..., 1.5452e-01,
            9.0802e-02, 5.8512e-01],
           [3.7022e-01, 7.5920e-02, 1.7262e-01,  ..., 1.2536e-01,
            1.8182e-01, 4.5449e-01],
           [0.0000e+00, 1.6054e-02, 3.6503e-02,  ..., 5.3587e-01,
            1.0942e-02, 9.3789e-01],
           ...,
           [1.1353e-02, 4.0257e-02,

In [29]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        print(batch_idx)
        print(data)
        
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        # if batch_idx % args.log_interval == 0:
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [30]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      #recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                                        recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [31]:
if __name__ == "__main__":
    #for epoch in range(1, args.epochs + 1):
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

ValueError: too many values to unpack (expected 2)